# Build your comparative framework!

In [6]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [5]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import seaborn as sns
from matplotlib import axes
import os 
import pandas as pd
import json

import sys, os
sys.path.append('/cluster/home/kamara/Explain')
from clutils.nbutils import *
from clutils.nbutils.params import get_param_ranges

In [8]:
pal = sns.color_palette("Paired", 11)
dict_color = {"random":pal[0], "distance":pal[1], "pagerank":pal[2], "sa": pal[3], "ig": pal[4], "gradcam": pal[5], "occlusion":pal[6], 
              "basic_gnnexplainer": pal[7], "gnnexplainer": pal[8], 
              "pgmexplainer":pal[9], "subgraphx": pal[10]}


In [9]:
sns.set()
sns.set_context("notebook", rc={"legend.fontsize":20, "legend.title_fontsize":20, 
                                "axes.titlesize":19,"axes.labelsize":17,
                               "xtick.labelsize" : 16, "ytick.labelsize" : 16})


In [10]:
dir = '/cluster/home/kamara/Explain/csv/node_classification/real'

jsonpath = f'/cluster/home/kamara/Explain/configs/config_real_sparsity_1expe.json'
ranges = get_param_ranges(jsonpath)

df_sparsity = pd.read_csv(os.path.join(dir, f'nc_real_sparsity_1expe.csv'), index_col = 0)
df_sparsity = df_sparsity.groupby(by=['dataset', 'sparsity', 'explainer_name', 'true_label_as_target', 'hard_mask']).mean()
df_sparsity = df_sparsity.reset_index()

In [11]:
def explainer_type(row):
    if row['explainer_name'] in ['basic_gnnexplainer', 'gnnexplainer', 'pgmexplainer', 'occlusion', 'subgraphx'] :
      return('pertubation')
    elif row['explainer_name'] in ['sa', 'ig', 'gradcam'] :
      return('gradient')
    return('baseline')

df_sparsity['type'] = df_sparsity.apply(lambda row: explainer_type(row), axis=1)

In [12]:
df_sparsity["1-fidelity_acc-"] = 1- df_sparsity["fidelity_acc-"]
df_sparsity["1-fidelity_prob-"] = 1- df_sparsity["fidelity_prob-"]
df_sparsity['charact_prob'] = 2 * df_sparsity['fidelity_prob+'] * (1-df_sparsity['fidelity_prob-'])/ (df_sparsity['fidelity_prob+']  + 1-df_sparsity['fidelity_prob-'])
df_sparsity['charact_acc'] = 2 * df_sparsity['fidelity_acc+'] * (1-df_sparsity['fidelity_acc-'])/ (df_sparsity['fidelity_acc+']  + 1-df_sparsity['fidelity_acc-'])


In [13]:
df_sparsity["1-fidelity_gnn_acc-"] = 1- df_sparsity["fidelity_gnn_acc-"]
df_sparsity["1-fidelity_gnn_prob-"] = 1- df_sparsity["fidelity_gnn_prob-"]
df_sparsity['charact_gnn_prob'] = 2 * df_sparsity['fidelity_gnn_prob+'] * (1-df_sparsity['fidelity_gnn_prob-'])/ (df_sparsity['fidelity_gnn_prob+']  + 1-df_sparsity['fidelity_gnn_prob-'])
df_sparsity['charact_gnn_acc'] = 2 * df_sparsity['fidelity_gnn_acc+'] * (1-df_sparsity['fidelity_gnn_acc-'])/ (df_sparsity['fidelity_gnn_acc+']  + 1-df_sparsity['fidelity_gnn_acc-'])


In [14]:
df_sparsity = df_sparsity.reset_index(drop=True)
df_sparsity

,dataset,sparsity,explainer_name,true_label_as_target,hard_mask,num_test,fidelity_acc+,fidelity_acc-,fidelity_prob+,fidelity_prob-,...,non_zero_values_init,type,1-fidelity_acc-,1-fidelity_prob-,charact_prob,charact_acc,1-fidelity_gnn_acc-,1-fidelity_gnn_prob-,charact_gnn_prob,charact_gnn_acc
0,actor,0.000,basic_gnnexplainer,False,False,100.0,-1.00,-1.00,-1.000000,-1.000000,...,1208.43,pertubation,2.00,2.000000,-4.000000,-4.000000,0.72,0.907846,0.267712,0.766753
1,actor,0.000,basic_gnnexplainer,False,True,100.0,-1.00,-1.00,-1.000000,-1.000000,...,1208.43,pertubation,2.00,2.000000,-4.000000,-4.000000,0.32,0.602857,0.299676,0.482462
2,actor,0.000,basic_gnnexplainer,True,False,100.0,0.77,0.38,0.190516,0.117405,...,1297.76,pertubation,0.62,0.882595,0.313385,0.686906,2.00,2.000000,-4.000000,-4.000000
3,actor,0.000,basic_gnnexplainer,True,True,100.0,0.96,0.62,0.236004,0.040103,...,1297.76,pertubation,0.38,0.959897,0.378860,0.544478,2.00,2.000000,-4.000000,-4.000000
4,actor,0.000,distance,False,False,100.0,-1.00,-1.00,-1.000000,-1.000000,...,60913.74,baseline,2.00,2.000000,-4.000000,-4.000000,0.21,0.838307,0.317123,0.345254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,wisconsin,0.997,random,True,True,100.0,0.24,0.89,0.112788,0.688925,...,1148.93,baseline,0.11,0.311075,0.165551,0.150857,2.00,2.000000,-4.000000,-4.000000
1435,wisconsin,0.997,sa,False,False,100.0,-1.00,-1.00,-1.000000,-1.000000,...,814.62,gradient,2.00,2.000000,-4.000000,-4.000000,0.33,0.428193,0.207677,0.186522
1436,wisconsin,0.997,sa,False,True,100.0,-1.00,-1.00,-1.000000,-1.000000,...,814.62,gradient,2.00,2.000000,-4.000000,-4.000000,0.45,0.542036,0.390044,0.321429
1437,wisconsin,0.997,sa,True,False,100.0,0.09,0.68,0.082403,0.598572,...,833.65,gradient,0.32,0.401428,0.136738,0.140488,2.00,2.000000,-4.000000,-4.000000


In [15]:
df = df_sparsity

In [16]:
sparsity_slider = widgets.SelectionSlider(
    options=['0', '0.7', '0.99', '0.997'],
    value='0',
    description='Sparsity:',
    disabled=False,
)



focus_select = widgets.ToggleButtons(
    options=[('Phenomenon', True), ('GNN model', False)],
    value = True,
    description='Focus:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Explain phenomenon', 'Explain GNN logic'],
#     icons=['check'] * 3
)

hard_mask_select = widgets.ToggleButtons(
    options=[('Hard', True), ('Soft', False)],
    value = True,
    description='Nature:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Mask has binary values', 'Mask has scalar values'],
#     icons=['check'] * 3
)

charact_type = widgets.ToggleButtons(
    options=[('Closeness', True), ('Reproducibility', False)],
    value = True,
    description='Characterization:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Characterization on probability', 'Characterization on accuracy'],
#     icons=['check'] * 3
)


dataset_select = widgets.Dropdown(
    options=["cora",
      "pubmed",
      "citeseer",
      "facebook",
      "cornell",
      "texas",
      "wisconsin",
      "actor",
      "chameleon",
      "squirrel"],
    value='cora',
    description='Dataset:',
    disabled=False,
)


explainer_select = widgets.Dropdown(
    options=["random",
      "distance",
      "pagerank",
      "sa",
      "ig",
      "gradcam",
      "occlusion",
      "basic_gnnexplainer",
      "gnnexplainer",
      "pgmexplainer"],
    value='random',
    description='Explainer:',
    disabled=False,
)


In [17]:
def reduce(df, **kwargs):
    if 'focus_phen' in kwargs:
        focus_phen = kwargs.get('focus_phen')
        df = df.loc[df['true_label_as_target'] == focus_phen]
        prob = kwargs.get('prob') if 'prob' in kwargs else True
        if focus_phen:
            df.loc[:, "charact"] = df.loc[:,"charact_prob"] if prob else df.loc[:,"charact_acc"]
            df.loc[:, "fid+"] = df.loc[:,"fidelity_prob+"] if prob else df.loc[:,"fidelity_acc+"]
            df.loc[:, "1-fid-"] = df.loc[:,"1-fidelity_prob-"] if prob else df.loc[:,"1-fidelity_acc-"]
        else:
            df.loc[:, "charact"] = df.loc[:, "charact_gnn_prob"] if prob else df.loc[:, "charact_gnn_acc"]
            df.loc[:, "fid+"] = df.loc[:, "fidelity_gnn_prob+"] if prob else df.loc[:, "fidelity_gnn_acc+"]
            df.loc[:, "1-fid-"] = df.loc[:, "1-fidelity_gnn_prob-"] if prob else df.loc[:, "1-fidelity_gnn_acc-"]
    
    if 'hard_mask' in kwargs:
        df = df.loc[df['hard_mask'] == kwargs.get('hard_mask')]
    if 'sparsity' in kwargs:
        df = df.loc[df['sparsity'] == eval(kwargs.get('sparsity'))]
    if 'dataset' in kwargs:
        df = df.loc[df['dataset'] == kwargs.get('dataset')]
    if 'explainer' in kwargs:
        df = df.loc[df['explainer_name'] == kwargs.get('explainer')]
    return df

In [18]:
@interact
def score(focus_phen=focus_select, hard_mask=hard_mask_select, sparsity=sparsity_slider, prob=charact_type):
    dfs = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask, sparsity=sparsity)
    df_char = pd.pivot_table(dfs, values='charact', index=['explainer_name'], columns='dataset')
    df_char['mean_charact'] = df_char.mean(axis=1)
    
    res = df_char.reset_index().rename_axis(None, axis=1)
    df_char = df_char.sort_values(by=['mean_charact'])
    
    sub = dfs.groupby('explainer_name').mean().reset_index().rename_axis(None, axis=1)
    sub = sub[["explainer_name", "mask_size_transf", "mask_entropy_transf", "max_avg_transf", '1-fid-', 'fid+', 'charact']]
    fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, sharey=True, figsize=(21, 10))
    
    sns.barplot(x="1-fid-", y="explainer_name", data=sub, palette=dict_color, ax=ax1)
    ax1.set(xlabel=None)
    ax1.set(ylabel="Performance")
    ax1.set_xlim(0,1)
    ax1.set(title="Causal")
    
    sns.barplot(x="fid+", y="explainer_name", data=sub, palette=dict_color, ax=ax2)
    ax2.set(xlabel=None)
    ax2.set(ylabel=None)
    ax2.set_xlim(0,1)
    ax2.set(title="Counterfactual")
    
    sns.barplot(x="charact", y="explainer_name", data=sub, palette=dict_color, ax=ax3)
    ax3.set(xlabel=None)
    ax3.set(ylabel=None)
    ax3.set_xlim(0,1)
    ax3.set(title="Characterisation") 
    
    sns.barplot(x="mask_size_transf", y="explainer_name", data=sub, palette=dict_color, ax=ax4)
    ax4.set(xlabel=None)
    ax4.set(ylabel="Properties")
    ax4.set(title="Selective")
    
    sns.barplot(x="mask_entropy_transf", y="explainer_name", data=sub, palette=dict_color, ax=ax5)
    ax5.set(xlabel=None)
    ax5.set(ylabel=None)
    ax5.set(title="Discriminative")
    
    sns.barplot(x="max_avg_transf", y="explainer_name", data=sub, palette=dict_color, ax=ax6)
    ax6.set(xlabel=None)
    ax6.set(ylabel=None)
    ax6.set(title="Confident") 
    
    fig.tight_layout()
    return

interactive(children=(ToggleButtons(description='Focus:', options=(('Phenomenon', True), ('GNN model', False))…

In [20]:
# Mask properties
@interact
def mask_properties(dataset=dataset_select, focus_phen=focus_select, hard_mask=hard_mask_select, prob=charact_type, sparsity=sparsity_slider):
    df_mask = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask, sparsity=sparsity, dataset=dataset)
    df_mask = df_mask[["explainer_name", "mask_size_transf", "mask_entropy_transf", "max_avg_transf", "charact"]]
    df_mask = df_mask.sort_values(by=['charact'])
    return df_mask.set_index('explainer_name')

interactive(children=(Dropdown(description='Dataset:', options=('cora', 'pubmed', 'citeseer', 'facebook', 'cor…

In [18]:
@interact
def get_explanation(focus_phen=focus_select, hard_mask=hard_mask_select, sparsity=sparsity_slider, prob=charact_type, dataset=dataset_select, explainer=explainer_select):
    df_unit = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask, sparsity=sparsity, dataset=dataset, explainer=explainer)
    # Causal: df_unit['1-fid-']
    # Counterfactual: df_unit['fid+']
    # Characterization: df_unit['charact']
    ax = sns.barplot(x="value", y="variable", data=pd.melt(df_unit[['1-fid-', 'fid+', 'charact']]), palette=sns.color_palette("tab10"))
    plt.xlim(0,1)
    ax.set_yticklabels(['causal','counterfactual', 'characterization'])
    ax.set(xlabel=None)
    ax.set(ylabel=None)
    ax.set(title="Type of explanation")
    
    
    textstr = '\n'.join((
    r'$\mathrm{selective}=%.2f$' % (df_unit['mask_size_transf'], ),
    r'$\mathrm{discriminative}=%.2f$' % (df_unit['mask_entropy_transf'], ),
    r'$\mathrm{confident}=%.2f$' % (df_unit['max_avg_transf'], )))

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    # place a text box in upper left in axes coords
    ax.text(1.05, 0.5, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)


interactive(children=(ToggleButtons(description='Focus:', options=(('Phenomenon', True), ('GNN model', False))…

In [16]:
@interact
def get_scatterplots_expl(focus_phen=focus_select, hard_mask=hard_mask_select, prob=charact_type):
    dfs = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask)
    # Causal: df_unit['1-fid-']
    # Counterfactual: df_unit['fid+']
    # Characterization: df_unit['charact']
    g = sns.relplot(
        data=dfs,
        x="fid+", y="1-fid-", col="sparsity", row="dataset", 
        hue="explainer_name", palette=dict_color, s=140, style="type", alpha =1, edgecolor="grey"
    )
    
    plt.xlim(0, 1)
    plt.ylim(0, 1.2)
    for lh in g._legend.legendHandles: 
        lh.set_alpha(1)
        lh._sizes = [130] 

interactive(children=(ToggleButtons(description='Focus:', options=(('Phenomenon', True), ('GNN model', False))…